---

## DCFC EVCS POI Candidate Selection - Atlanta

**Atlanta**: Since a larger number of DCFC-type EVCS (Electric Vehicle Charging Stations) are installed in Atlanta compared to other regions, an initial **POI filtering** was conducted to identify potential locations for DCFC installation. Additionally, to further refine the placement of DCFC EVCS, **case-specific optimization** was applied. This process first involved running an optimization for **Level 2 EVCS** installations. The results from this optimization were then used as additional candidates for DCFC installation.

---

### Summary
  
- **DCFC POI Candidate Selection**: Atlanta, using GIS tools, filtering, and optimization of Level 2 EVCS results.  

-  **POI Filtering Results** + **Optimization of Level 2 EVCS** = Final **DCFC EVCS Candidates** for Atlanta.  
<br>
---


# DCFC POI Candidate Selection - Atlanta

In [3]:
import yaml
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from shapely.geometry import box, Point
from rasterio.mask import geometry_mask

from src import setup_logging, greedy_optimization, process_ev_charging_data

In [ ]:
# 1. Load the YAML file with fclass categories
yaml_file_path = 'poi_filtering.yaml'

with open(yaml_file_path, 'r') as file:
    fclass_data = yaml.safe_load(file)

# 2. Extract the list of categories for 'atlanta_dcfc' from the YAML data
if 'atlanta_dcfc' in fclass_data['candidate']:
    selected_category = fclass_data['candidate']['atlanta_dcfc']
else:
    raise ValueError("The 'atlanta_dcfc' category is not found in the YAML file.")

# 3. File paths (Modify these paths as per your file locations)
gpkg_file_path = "Path to your Atlanta Region POI"
result_of_atlanta = 'Path to Optimization Result in Atlanta' 

# 4. Read the GPKG files
poi_gdf = gpd.read_file(gpkg_file_path)
poi2_gdf = gpd.read_file(result_of_atlanta)

# 5. Filter rows from the 'fclass' column that match the selected categories
poi_gdf = poi_gdf[poi_gdf['fclass'].isin(selected_category)]

# 6. Prepare the second GeoDataFrame
poi2_gdf = poi2_gdf[['osm_id', 'fclass', 'geometry']]

# 7. Merge the two GeoDataFrames and remove duplicates based on 'osm_id'
merged = pd.concat([poi_gdf, poi2_gdf], ignore_index=True)
merged = merged.drop_duplicates(subset='osm_id')

# 8. Convert the merged DataFrame back to a GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged, geometry='geometry')

# 9. Save the merged GeoDataFrame to a new GeoPackage file
output_file_path = "atlanta_dcfc_poi_candidate.gpkg"
merged_gdf.to_file(output_file_path, driver="GPKG")

print(f"Filtered and merged POI data saved to {output_file_path}")

---

## Case 2: Atlanta DCFC EVCS Charger Initial Point Selection

The **Atlanta** area is a key region for the installation of numerous **DCFC** (Direct Current Fast Chargers).  
Since the optimal location of DCFCs can be critical to traffic flow, we use the **road network** to guide the selection of initial points.

### Algorithm:

1. **Map demand values** using the **Motorway** and **Trunkway Demand Map**. Start by selecting POIs adjacent to the roads with the highest demand values based on the following criteria:
2. Find **POI candidates** near roads with high demand values, and ensure that the selected POIs maintain a **minimum distance of 3 km**, which is the minimum service radius for EVCS.
3. Select the specified number of **initial POIs**, repeating the process until the required number of points is reached.
4. If fewer than 5 POIs are selected in a specific area, the process reads the **Demand Map's pixel values** directly to identify high-demand regions and select additional POI candidates.  
<br>  
---

In [ ]:
# Load datasets
def load_data():
    motorway_gdf = gpd.read_file('Path_to_Motorway_Road_Network.gpkg')  # Load motorway road network
    trunk_gdf = gpd.read_file('Path_to_Trunk_Road_Network.gpkg')  # Load trunk road network
    poi_gdf = gpd.read_file('Path_to_DCFC_Candidate_POIs.gpkg')  # Load candidate POIs for DCFC
    demand_map = rasterio.open('Path_to_Demand_Map.tif')  # Load demand map raster
    boundary_gdf = gpd.read_file('Path_to_Boundary_Polygon.gpkg')  # Load boundary polygon data
    return motorway_gdf, trunk_gdf, poi_gdf, demand_map, boundary_gdf

# Extract pixel values from demand map based on geometry
def extract_demand_values(geometries, demand_map):
    demand_values = []
    for geom in geometries:
        mask = geometry_mask([geom], transform=demand_map.transform, invert=True, out_shape=(demand_map.height, demand_map.width))
        pixel_data = demand_map.read(1, masked=True)[mask]
        if pixel_data.size > 0:
            demand_values.append((geom, np.max(pixel_data)))
        else:
            demand_values.append((geom, None))
    return demand_values

# Calculate Euclidean distance in meters (EPSG:3857 is a meter-based projection)
def calculate_euclidean_distance(point1, point2):
    return point1.distance(point2)

# Check if new POI is far enough from existing selected POIs
def check_min_distance(selected_pois, new_poi_geom, min_distance_m=3000):
    for existing_poi in selected_pois.itertuples():
        if calculate_euclidean_distance(existing_poi.geometry, new_poi_geom) < min_distance_m:
            return False
    return True

# Select initial POIs for each region based on demand and proximity
def select_initial_pois_for_region(region_boundary, motorway_gdf, trunk_gdf, poi_gdf, demand_map, target_crs):
    # Filter roads within region
    motorway_in_region = motorway_gdf[motorway_gdf.intersects(region_boundary.unary_union)]
    trunk_in_region = trunk_gdf[trunk_gdf.intersects(region_boundary.unary_union)]

    # Combine road networks into one GeoDataFrame
    combined_roads_gdf = gpd.GeoDataFrame(pd.concat([motorway_in_region, trunk_in_region], ignore_index=True))

    # Get demand values for road geometries
    road_demand_values = extract_demand_values(combined_roads_gdf.geometry, demand_map)
    road_demand_values = [v for v in road_demand_values if v[1] is not None]
    road_demand_values.sort(key=lambda x: x[1], reverse=True)  # Sort by demand (descending)

    # Select POIs based on demand and proximity
    selected_pois_gdf = gpd.GeoDataFrame(columns=poi_gdf.columns, crs=target_crs)
    for road_geom, _ in road_demand_values:
        candidates_in_area = poi_gdf[poi_gdf.intersects(road_geom)]
        for _, candidate_poi in candidates_in_area.iterrows():
            candidate_poi_gdf = gpd.GeoDataFrame([candidate_poi], crs=poi_gdf.crs)
            if len(selected_pois_gdf) > 0 and not check_min_distance(selected_pois_gdf, candidate_poi_gdf.geometry.iloc[0]):
                continue
            selected_pois_gdf = pd.concat([selected_pois_gdf, candidate_poi_gdf], ignore_index=True)
            if len(selected_pois_gdf) >= 5:
                break
        if len(selected_pois_gdf) >= 5:
            break

    # Handle regions with fewer than 5 POIs using demand map directly
    if len(selected_pois_gdf) < 5:
        pixel_data = demand_map.read(1)
        sorted_pixel_indices = np.unravel_index(np.argsort(pixel_data, axis=None)[::-1], pixel_data.shape)
        for y, x in zip(sorted_pixel_indices[0], sorted_pixel_indices[1]):
            pixel_geom = box(*demand_map.xy(y, x), *demand_map.xy(y + 1, x + 1))
            if region_boundary.unary_union.intersects(pixel_geom):
                candidates_in_area = poi_gdf[poi_gdf.intersects(pixel_geom)]
                for _, candidate_poi in candidates_in_area.iterrows():
                    candidate_poi_gdf = gpd.GeoDataFrame([candidate_poi], crs=poi_gdf.crs)
                    if len(selected_pois_gdf) > 0 and not check_min_distance(selected_pois_gdf, candidate_poi_gdf.geometry.iloc[0]):
                        continue
                    selected_pois_gdf = pd.concat([selected_pois_gdf, candidate_poi_gdf], ignore_index=True)
                    if len(selected_pois_gdf) >= 5:
                        break
            if len(selected_pois_gdf) >= 5:
                break

    return selected_pois_gdf

# Main logic to select POIs for each region
def select_pois_for_all_regions(boundary_gdf, motorway_gdf, trunk_gdf, poi_gdf, demand_map, target_crs):
    all_selected_pois = []
    for region_id in boundary_gdf['fid'].unique():
        region_boundary = boundary_gdf[boundary_gdf['fid'] == region_id]
        selected_pois_gdf = select_initial_pois_for_region(region_boundary, motorway_gdf, trunk_gdf, poi_gdf, demand_map, target_crs)
        all_selected_pois.append(selected_pois_gdf)

    # Combine all selected POIs into a single GeoDataFrame
    if all_selected_pois:
        return gpd.GeoDataFrame(pd.concat(all_selected_pois, ignore_index=True), crs=target_crs)
    return None

# Save selected POIs to file
def save_selected_pois(selected_pois_gdf, output_file):
    if selected_pois_gdf is not None:
        selected_pois_gdf.to_file(output_file, driver="GPKG")
        print(f"Total selected POIs: {len(selected_pois_gdf)}")
    else:
        print("No POIs were selected.")

# Example of usage in a Jupyter notebook environment
motorway_gdf, trunk_gdf, poi_gdf, demand_map, boundary_gdf = load_data()
target_crs = 'EPSG:3857'  # Using EPSG:3857 which is meter-based and suitable for Euclidean distance

# Select POIs for all regions
selected_pois_gdf = select_pois_for_all_regions(boundary_gdf, motorway_gdf, trunk_gdf, poi_gdf, demand_map, target_crs)

# Save the results
save_selected_pois(selected_pois_gdf, 'atlanta_dcfc_initial.gpkg')

---

## Optimization

In [2]:
setup_logging()

gpkg_file = "/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/data/for_model/urban_trip_DCFC_greedy.gpkg"
tif_file = "/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/data/demand_map_500.tif"
poi_file = "/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/data/for_model/Atlanta_Final_DCFAST_Candidate.gpkg"
output_path = "./results/"

# Read input files
polygons = gpd.read_file(gpkg_file)
poi_gdf = gpd.read_file(poi_file)

for _, polygon in polygons.iterrows(): 
    greedy_optimization(
        polygon, 
        tif_file, 
        poi_gdf, 
        capture_range=3000, 
        bandwidth=1000, 
        constraints=(1, None), 
        output_path=output_path,
        save_intermediate=True
    )

2024-09-29 20:54:03,906 - INFO - 
*** Processed Polygon Information ***
--------------------------------------------------
Polygon ID            : Atlanta, GA Urban Area_6
Total Supply          : 176.0
A_bar Value           : 0.0014
Selected Initial Sites: 5
Initial Coverage      : 15.42%
2024-09-29 20:54:09,869 - INFO - Selecting site  6/35 | Selected Site: 132 | A_hat: 0.00400 | Coverage:  15.42%
2024-09-29 20:54:14,816 - INFO - Selecting site  7/35 | Selected Site: 107 | A_hat: 0.00369 | Coverage:  16.47%
2024-09-29 20:54:18,011 - INFO - Selecting site  8/35 | Selected Site:   5 | A_hat: 0.00344 | Coverage:  19.11%
2024-09-29 20:54:21,789 - INFO - Selecting site  9/35 | Selected Site: 102 | A_hat: 0.00323 | Coverage:  22.32%
2024-09-29 20:54:26,077 - INFO - Selecting site 10/35 | Selected Site: 139 | A_hat: 0.00307 | Coverage:  25.20%
2024-09-29 20:54:30,941 - INFO - Selecting site 11/35 | Selected Site: 208 | A_hat: 0.00293 | Coverage:  27.44%
2024-09-29 20:54:34,134 - INFO - Selec

KeyboardInterrupt: 